<h1>Ensemble - Bagging</h1>

Ensemble bedeutet, dass mehrere Algorithmen für eine Antwort verwendet werden, z. B. als Mehrheitsentscheidung wie bei Random-Forest.

Bei Random-Forest wird der Algorithmus für Entscheidungsbäume genommen, die mehrere Bäume erzeugen und mehrere Antworten liefern. Durch eine Mehrheitsentscheidung gibt es am Ende nur eine Antwort, genau das ist Esemble Learning.

Ensemble Learning bietet zwei Methoden an-
- Bagging (Bootstrap Aggregation)
- Boosting

Diese können bei Modellen eingesetzt werden, wenn Datasets durch deren Struktur, Eigenschaften, Aufteilung, ..., dafür sorgen, das das Model beim Trainieren gut punktet, aber beim Testen eine hohe Varianz aufweist. <br>
Hohe Varianz eines Models (Test errors) => Indikator für Overfitting. 

<i>Abb1</i>: Bagging - Resampling with Replacement. 

<img src="./files_data/img/sklearn_ensemble_1.PNG" width=650 hight=450> 

Aus einem gegebenen Dataset werden n-Samples zufällig gezogen. Die Wahrscheinlichkeit der Ziehung ist gleich verteilt, und es können Duplikate vorkommen. Dieser Vorgang mit Resampling with replacement nennt sich Bootstrap. 

Es können n-Subsets eines Datasets erstellt werden, die dann je ein Model bedienen. Genau so funktioniert auch Random-Forest.
Jeder dieser Modelle hat unterschiedliche Subsets gefüllt mit anderen Daten, was auch zu anderen Antworten führt. Durch eine Mehrheitsentscheidung wird festgestellt, welche Antwort genommen wird => z. B. True / False

Jeder der Sub-Modelle führen eine Prediction aus. Jeder dieser Sub-Modelle wird als weak learner bezeichnet (weil mit Subset trainiert).

<i>Abb2</i>: Mehrheitsentscheidung bei Bagging, Gesamtüberblick. 

<img src="./files_data/img/sklearn_ensemble_2.PNG" width=650 hight=650> 

Die kombinierten Ergebnisse der einzelnen Sub-Modelle liefern allgemein gute Ergebnisse.
- Bei Regression wird der Durchschnitt der Antworten genommen.

Random-Forest an sich ist ein Model, das schon Bagging nutzt und auch die Features sampelt. Jedes Subset bildet einen Baum. <br>
Ein einziger Baum, oder ein einzelnes Model kann zu Overfitting führen => keine gute Balance. <br>
Mehrere Modelle können das Dataset aus verschiedenen Winkeln betrachten und zusammen ein besseres Ergebnis liefern.

Für die Anwendung von Bagging nutzen wir ein Heart-Failure Dataset und SVM, dann wird der Score mit und ohne Bagging verglichen.

In [25]:
# Imports
import pandas as pd   

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm  import SVC

from sklearn.ensemble import BaggingClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing   import StandardScaler  
from sklearn.preprocessing   import LabelEncoder

In [8]:
# Lade das Dataset in ein Dataframe.
heart_data = pd.read_csv("./files_data/data/heart_failure_prediction.zip", compression='zip')
heart_data.head(2)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1


In [9]:
# Gibt es Null-Werte?
heart_data.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

Ohne weiter ins Detail zu gehen, werden die Features encoded und skaliert.  <br>

In [10]:
# On Hot Encode mit Pandas
dummies_df = pd.get_dummies(heart_data[['Sex', 'ExerciseAngina']], dtype="int")
# Lösche betroffene Spalten.
heart_data.drop(['Sex', 'ExerciseAngina'], axis='columns', inplace=True)
# Hänge neue Spalten an.
heart_data = pd.concat([dummies_df, heart_data ], axis='columns')

# Erstelle Sklearn Label Encoder.
le = LabelEncoder()
# Encode Features.
heart_data['ChestPainType']  = le.fit_transform(heart_data['ChestPainType'])
heart_data['RestingECG']     = le.fit_transform(heart_data['RestingECG'])
heart_data['ST_Slope']       = le.fit_transform(heart_data['ST_Slope'])

heart_data.head(5)

,Sex_F,Sex_M,ExerciseAngina_N,ExerciseAngina_Y,Age,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,Oldpeak,ST_Slope,HeartDisease
0,0,1,1,0,40,1,140,289,0,1,172,0.0,2,0
1,1,0,1,0,49,2,160,180,0,1,156,1.0,1,1
2,0,1,1,0,37,1,130,283,0,2,98,0.0,2,0
3,1,0,0,1,48,0,138,214,0,1,108,1.5,1,1
4,0,1,1,0,54,2,150,195,0,1,122,0.0,2,0


Optional können die Daten skaliert werden. 

In [12]:
scaler = StandardScaler()
X_data = scaler.fit_transform(heart_data.drop(['HeartDisease'], axis="columns"))
X_data.shape

(918, 13)

In [19]:
# Erstelle Model.
svc = SVC()

cross_val_score(svc, X_data, heart_data['HeartDisease'], cv=4)

array([0.86956522, 0.9       , 0.83842795, 0.75982533])

Mit Cross Validation 4 ist zu sehen, dass der Score sich je nach Aufteilung des Datasets im Score unterscheidet.
- Model hat eine höhere Varianz

Was machen könnte, ist Bagging zu verwenden (was auch den Anwendungszeck zeigen soll, deswegen haben wir das Model nicht optimiert...).

In [24]:
# Dasselbe kann auch mit einem Baum gemacht werden. 
# - Auch eine höhere Varianz. 
dtree = DecisionTreeClassifier()

cross_val_score(dtree, X_data, heart_data['HeartDisease'], cv=4)

array([0.8       , 0.83478261, 0.79039301, 0.68995633])

In [42]:
# Einige Parameter, die eingestellt werden können
# - Siehe Sklearn Doks. 
model = BaggingClassifier( 
    estimator= svc,        # Welches Model. 
    n_estimators  = 10,    # Wie viele Modelle => n-Subsets -> n-Modelle
    max_samples   = 0.85,  # Pool für Subsets => 100 Samples und max_samples=0.85 -> Subset hat 85 Samples. 
    oob_score     = True   # Out of Bag. Wenn Datenpunk nicht gezogen wird => Nutze es als Testset. Nur bei Bootstrap. 
    # verbose       = 2    # Zusätzliche Informationen => kann viel Text generieren
)
model

BaggingClassifier(estimator=SVC(), max_samples=0.85, oob_score=True)

In [43]:
# Trainiere.
model.fit(X_data, heart_data['HeartDisease'])
model.oob_score_

C:\Users\Alexander\anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:789: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
C:\Users\Alexander\anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:795: RuntimeWarning: invalid value encountered in divide
  oob_decision_function = predictions / predictions.sum(axis=1)[:, np.newaxis]


0.8747276688453159

In [47]:
# SVM
model = BaggingClassifier( 
    estimator= svc,        # Welches Model. 
    n_estimators  = 20,    # Wie viele Modelle => n-Subsets -> n-Modelle
    max_samples   = 0.85,  # Pool für Subsets => 100 Samples und max_samples=0.85 -> Subset hat 85 Samples. 
    #oob_score     = True  # Out of Bag. Wenn Datenpunk nicht gezogen wird => Nutze es als Testset. Nur bei Bootstrap. 
    # verbose       = 2    # Zusätzliche Informationen => kann viel Text generieren
)

cross_val_score(model, X_data, heart_data['HeartDisease'], cv=4)

array([0.85652174, 0.91304348, 0.86899563, 0.73799127])

In [48]:
# Baum 
model = BaggingClassifier( 
    estimator= dtree,      # Welches Model. 
    n_estimators  = 20,    # Wie viele Modelle => n-Subsets -> n-Modelle
    max_samples   = 0.85,  # Pool für Subsets => 100 Samples und max_samples=0.85 -> Subset hat 85 Samples. 
    #oob_score     = True  # Out of Bag. Wenn Datenpunk nicht gezogen wird => Nutze es als Testset. Nur bei Bootstrap. 
    # verbose       = 2    # Zusätzliche Informationen => kann viel Text generieren
)

cross_val_score(model, X_data, heart_data['HeartDisease'], cv=4)

array([0.86956522, 0.87391304, 0.8209607 , 0.72489083])

Zusammengefasst:

In [49]:
# SVM Score # 
svc = SVC()
cross_val_score(svc, X_data, heart_data['HeartDisease'], cv=4).mean()

0.8419546231251187

In [56]:
# SVM Score mit Bagging # 
model = BaggingClassifier( 
    estimator= svc,       
    n_estimators  = 10,   
    max_samples   = 0.78,  
)
cross_val_score(model, X_data, heart_data['HeartDisease'], cv=4).mean()

0.8420479302832243

In [57]:
# Baum #
dtree = DecisionTreeClassifier()
cross_val_score(dtree, X_data, heart_data['HeartDisease'], cv=4).mean()

0.7766138219100057

In [58]:
# Baum mit Bagging # 
model = BaggingClassifier( 
    estimator= dtree,      
    n_estimators  = 20,    
    max_samples   = 0.85,
)

cross_val_score(model, X_data, heart_data['HeartDisease'], cv=4).mean()

0.8158202012530852

In [62]:
# Baum mit Bagging und Feature sampling (wie Random-Forest) # 
model = BaggingClassifier( 
    estimator= dtree,      
    n_estimators  = 20,    
    max_samples   = 0.85,   
    max_features  = 0.90  # Feature pool. 
)

cross_val_score(model, X_data, heart_data['HeartDisease'], cv=4).mean()

0.8256265426238846

In [ ]:
# // Content Coming [Boosting]